# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770855874654         NaN   2.97e-01   0.80    9.0
  2   -2.772141382747   -1.29e-03   5.02e-02   0.80    1.0
  3   -2.772170139747   -2.88e-05   2.03e-03   0.80    1.0
  4   -2.772170721706   -5.82e-07   9.08e-05   0.80    2.0
  5   -2.772170723003   -1.30e-09   1.06e-05   0.80    2.0
n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770843076421         NaN   2.99e-01   0.80    9.0
  2   -2.772061769647   -1.22e-03   4.78e-02   0.80    1.0
  3   -2.772082737838   -2.10e-05   4.05e-03   0.80    2.0
  4   -2.772083417577   -6.80e-07   1.12e-04   0.80    2.0
  5   -2.772083417801   -2.24e-10   1.79e-05   0.80    1.0


1.7734642692446656

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770763440423         NaN   3.00e-01   0.80    9.0
  2   -2.772059643973   -1.30e-03   4.71e-02   0.80    1.0
  3   -2.772082319334   -2.27e-05   4.77e-03   0.80    2.0
  4   -2.772083417475   -1.10e-06   1.09e-04   0.80    2.0
  5   -2.772083417798   -3.24e-10   1.62e-05   0.80    2.0

Polarizability via ForwardDiff:       1.7725485552573657
Polarizability via finite difference: 1.7734642692446656
